### Due to GitHub's file size limit (maximum 100MB per file), the larger RoBERTa-based models are not included in this repository.

Specifically, files under ./models/* and ./packages/* are excluded from version control.

### 由于 GitHub 对单个文件大小的限制（最大 100MB），基于 RoBERTa 的大规模模型未包含在此仓库中。

具体而言，./models/* 与 ./packages/* 路径下的文件未纳入版本控制。

In [1]:
# =========================
# Cell 0：基础导入 & 路径设置
# =========================
from pathlib import Path
import os
import spacy
from spacy.lookups import Lookups
from spacy.tokens import DocBin
from spacy.cli.package import package

# 项目根目录
project_root = Path(".").resolve()

# 目录结构
corpus_dir = project_root / "corpus"
models_dir = project_root / "models"
model_name = "lv_spacy_cnn"
trained_model_path = models_dir / model_name / "model-best"
final_model_path = models_dir / model_name / "model_lv_cnn"
lookups_path = project_root / "lookups_lv"
package_output_dir = project_root / "packages"
config_path = project_root / "config" / "config_spacy_cnn.cfg"

# 创建目录
for p in [corpus_dir, models_dir, models_dir / model_name, package_output_dir, lookups_path, project_root / "config"]:
    p.mkdir(parents=True, exist_ok=True)

print("✅ 目录结构准备完成")



✅ 目录结构准备完成


In [2]:
# ===============================
# Cell 1：Conllu 转换为 spaCy 格式
# ===============================
!python -m spacy convert ud_latvian/lv_lvtb-ud-train.conllu ./corpus -n 10
!python -m spacy convert ud_latvian/lv_lvtb-ud-dev.conllu ./corpus -n 10
!python -m spacy convert ud_latvian/lv_lvtb-ud-test.conllu ./corpus -n 10


ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1506 documents):
corpus/lv_lvtb-ud-train.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (208 documents):
corpus/lv_lvtb-ud-dev.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (240 documents):
corpus/lv_lvtb-ud-test.spacy


In [3]:
# ========================================================
# Cell 2：初始化 config （不要开GPU，否则会变成transformer架构）
# ========================================================
!python -m spacy init config ./config/config_spacy_cnn.cfg \
    --lang lv \
    --pipeline tok2vec,tagger,morphologizer,parser,senter \
    --optimize efficiency



ℹ Generated config template specific for your use case
- Language: lv
- Pipeline: tagger, morphologizer, parser, senter
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config/config_spacy_cnn.cfg
You can now add your data and train your pipeline:
python -m spacy train config_spacy_cnn.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [4]:
# ==========================
# Cell 3：修改 config
# ==========================

cfg_text = config_path.read_text(encoding="utf-8")

# 替换训练/验证集路径
cfg_text = cfg_text.replace("train = null", f"train = {corpus_dir}/lv_lvtb-ud-train.spacy")
cfg_text = cfg_text.replace("dev = null", f"dev = {corpus_dir}/lv_lvtb-ud-dev.spacy")

# 开启混合精度
cfg_text = cfg_text.replace("mixed_precision = false", "mixed_precision = true")

config_path.write_text(cfg_text, encoding="utf-8")
print("✅ config 已更新训练/验证路径, 并启用混合精度")


✅ config 已更新训练/验证路径, 并启用混合精度


In [5]:
# ================================
# Cell 4：生成 lemma lookup table
# ================================

# 加载训练集
docbin = DocBin().from_disk(corpus_dir / "lv_lvtb-ud-train.spacy")
lemma_dict = {}

for doc in docbin.get_docs(spacy.blank("lv").vocab):
    for token in doc:
        if token.lemma_:
            lemma_dict[token.text.lower()] = token.lemma_

lookups = Lookups()
lookups.add_table("lemma_lookup", lemma_dict)
lookups.to_disk(lookups_path)
print(f"✅ 已生成 lemma lookup table, 路径: {lookups_path}")


/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 已生成 lemma lookup table, 路径: /home/jesse/Projects/myprojs/spaCy_Pipeline/LV_spacy_cnn/lookups_lv


In [6]:
# =============================
# Cell 5：训练 spaCy tok2vec 模型
# =============================
!python -m spacy train ./config/config_spacy_cnn.cfg \
    --output ./models/lv_spacy_cnn\
    --paths.train ./corpus/lv_lvtb-ud-train.spacy \
    --paths.dev ./corpus/lv_lvtb-ud-dev.spacy \
    --gpu-id 0


ℹ Saving to output directory: models/lv_spacy_cnn
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'tagger', 'morphologizer', 'parser',
'senter']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TAGGER  LOSS MORPH...  LOSS PARSER  LOSS SENTER  TAG_ACC  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_P  SENTS_R  SENTS_F  SCORE 
---  ------  ------------  -----------  -------------  -----------  -----------  -------  -------  ---------  -------  -------  -------  -------  -------  ------
  0       0          0.00       202.94         202.86       447.71       112.50    18.53    31.64      21.10     8.67     6.82     0.01     0.19     0.02    0.13
  0     200       3277.27     23148.57       21820.15     33025.80      2844.51    51.81    77.86      58.26    51.50    33.52    56.32    64.90    60.31    0.56
  0    

In [7]:
# ========================
# Cell 6: 模型评估
# ========================
!python -m spacy evaluate ./models/lv_spacy_cnn//model-best ./corpus/lv_lvtb-ud-test.spacy --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK      99.53
TAG      86.61
POS      94.79
MORPH    90.73
UAS      80.76
LAS      75.24
SENT P   96.27
SENT R   95.83
SENT F   96.05
SPEED    23939


============================== MORPH (per feat) ==============================

                P       R       F
ExtPos      90.00   84.17   86.99
Case        92.02   91.67   91.85
Gender      93.41   92.96   93.18
Number      93.87   92.99   93.43
Person      95.27   93.79   94.52
PronType    98.59   98.00   98.30
Evident     94.62   92.71   93.65
Mood        93.72   91.90   92.80
Polarity    94.38   92.65   93.51
Tense       91.41   88.82   90.10
VerbForm    93.04   91.31   92.17
Voice       93.95   91.36   92.64
Definite    90.39   87.74   89.05
Degree      93.10   90.73   91.90
Poss       100.00   98.91   99.45
NumType     98.66   77.06   86.53
Reflex      93.27   92.37   92.81
Aspect      89.62   85.40   87.46
Foreign     83.47   53.16  

In [8]:
# ==========================================
# Cell 7：添加 Lemmatizer (lookup) 并保存模型
# ==========================================
import spacy
from spacy.lookups import Lookups
from pathlib import Path

trained_model_path = "./models/lv_spacy_cnn/model-best"
final_model_path = "./models/lv_spacy_cnn/model_lv_cnn"
lookups_path = "./lookups_lv"

# 1️⃣ 加载训练好的模型
nlp = spacy.load(trained_model_path)

# 2️⃣ 加载 lookups
lookups = Lookups().from_disk(lookups_path)

# 3️⃣ 添加 lemmatizer（lookup 模式）并直接传入 lookups
lemmatizer = nlp.add_pipe("lemmatizer", config={"mode":"lookup"}, last=True)
lemmatizer.lookups = lookups  # ⚡ 注意这里直接赋值属性，而不是在 config 里传

# 4️⃣ 保存最终模型
nlp.to_disk(final_model_path)
print(f"✅ 模型已保存到 {final_model_path}，包含 lemmatizer + lookups")


✅ 模型已保存到 ./models/lv_spacy_cnn/model_lv_cnn，包含 lemmatizer + lookups


In [9]:
# =======================
# Cell 8：打包模型
# =======================

from spacy.cli.package import package
from pathlib import Path
import os

project_root = Path(".").resolve()
os.environ["PYTHONPATH"] = f"{project_root}:{os.environ.get('PYTHONPATH','')}"

# ⚡ 注意这里把字符串路径换成 Path 对象
package(
    input_dir=Path(final_model_path),
    output_dir=Path(package_output_dir),
    name="spacy_cnn",
    version="1.0.0",
    force=True
)

print(f"✅ 打包完成，发布包在 {package_output_dir} 文件夹里")

* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for sdist...
running egg_info
creating lv_spacy_cnn.egg-info
writing lv_spacy_cnn.egg-info/PKG-INFO
writing dependency_links to lv_spacy_cnn.egg-info/dependency_links.txt
writing entry points to lv_spacy_cnn.egg-info/entry_points.txt
writing top-level names to lv_spacy_cnn.egg-info/top_level.txt
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
* Building sdist...


running sdist
running egg_info
writing lv_spacy_cnn.egg-info/PKG-INFO
writing dependency_links to lv_spacy_cnn.egg-info/dependency_links.txt
writing entry points to lv_spacy_cnn.egg-info/entry_points.txt
writing top-level names to lv_spacy_cnn.egg-info/top_level.txt
reading manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
running check
creating lv_spacy_cnn-1.0.0
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn.egg-info
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/lemmatizer/lookups
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/parser
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/senter
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/tagger
creating lv_spacy_cnn-1.0.0/lv

In [10]:
# =======================
# Cell 9：生成 wheel + sdist
# =======================
import subprocess
from pathlib import Path

package_output_dir = Path("./packages/lv_spacy_cnn-1.0.0")

# 进入打包目录生成 wheel 和 sdist
subprocess.run(
    ["python", "-m", "build", "--wheel", "--sdist"],
    cwd=str(package_output_dir)
)

print(f"✅ wheel + sdist 已生成，路径: {package_output_dir / 'dist'}")


* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for wheel...
running egg_info
writing lv_spacy_cnn.egg-info/PKG-INFO
writing dependency_links to lv_spacy_cnn.egg-info/dependency_links.txt
writing entry points to lv_spacy_cnn.egg-info/entry_points.txt
writing top-level names to lv_spacy_cnn.egg-info/top_level.txt
reading manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
* Building wheel...


running bdist_wheel
running build
running build_py
creating build/lib/lv_spacy_cnn
copying lv_spacy_cnn/__init__.py -> build/lib/lv_spacy_cnn
creating build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/meta.json -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/README.md -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/tokenizer -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/config.cfg -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
creating build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer/cfg -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer/model -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
creating build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0/tagger
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/tagger/cfg -> build/lib/lv_

running sdist
running egg_info
writing lv_spacy_cnn.egg-info/PKG-INFO
writing dependency_links to lv_spacy_cnn.egg-info/dependency_links.txt
writing entry points to lv_spacy_cnn.egg-info/entry_points.txt
writing top-level names to lv_spacy_cnn.egg-info/top_level.txt
reading manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
running check
creating lv_spacy_cnn-1.0.0
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn.egg-info
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/lemmatizer/lookups
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/parser
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/senter
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/tagger
creating lv_spacy_cnn-1.0.0/lv

In [11]:
# ==========================
# Cell 10A: 通过wheel文件安装
# ==========================
import subprocess
import spacy
import pandas as pd


# ⚡ 安装 wheel
subprocess.run([
    "pip", 
    "install", 
    "./packages/lv_spacy_cnn-1.0.0/dist/lv_spacy_cnn-1.0.0-py3-none-any.whl"
])

# 加载模型
nlp_xlmr = spacy.load("lv_spacy_cnn")

print("lv_spacy_cnn Pipeline组件:", nlp_xlmr.pipe_names)

Processing ./packages/lv_spacy_cnn-1.0.0/dist/lv_spacy_cnn-1.0.0-py3-none-any.whl
lv_spacy_cnn Pipeline组件: ['tok2vec', 'tagger', 'morphologizer', 'parser', 'senter', 'lemmatizer']


In [12]:
# =============================
# Cell 10B: 通过'tar.gz'文件安装
# =============================
import subprocess
import spacy
import pandas as pd

# 安装 '.tar.gz'
subprocess.run(["pip", "install", "./packages/lv_spacy_cnn-1.0.0/dist/lv_spacy_cnn-1.0.0.tar.gz"])
nlp_xlmr = spacy.load("lv_spacy_cnn")

print("lv_spacy_cnn Pipeline组件:", nlp_xlmr.pipe_names)

Processing ./packages/lv_spacy_cnn-1.0.0/dist/lv_spacy_cnn-1.0.0.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  DEPRECATION: Building 'lv_spacy_cnn' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'lv_spacy_cnn'. Discussion can be found at https://github.com/pypa/pip/issues/6334


  Created wheel for lv_spacy_cnn: filename=lv_spacy_cnn-1.0.0-py3-none-any.whl size=9420601 sha256=21eb5140086a1a63cbb6e342fe1f17c87969f765160dc1dadc03705d057629f3
  Stored in directory: /home/jesse/.cache/pip/wheels/2d/1a/c9/7bf55f11beaa06f43598de80d2227919a05fe67861d49051e8
Successfully built lv_spacy_cnn
  Attempting uninstall: lv_spacy_cnn
    Found existing installation: lv_spacy_cnn 1.0.0
    Uninstalling lv_spacy_cnn-1.0.0:
      Successfully uninstalled lv_spacy_cnn-1.0.0
lv_spacy_cnn Pipeline组件: ['tok2vec', 'tagger', 'morphologizer', 'parser', 'senter', 'lemmatizer']


In [13]:
# ========================================
# Cell 11: 测试模型，显示 Lemma，并显示分句
# ========================================
import spacy
import pandas as pd

nlp = spacy.load("lv_spacy_cnn")

text = """Baltijas jūras nosaukums ir devis nosaukumu baltu valodām un Baltijas valstīm. Terminu "Baltijas jūra" (Mare Balticum) pirmoreiz lietoja vācu hronists Brēmenes Ādams 11. gadsimtā. Viņš, kā jau vietējais hronists, pierakstījis nosaukumu, kuru lietoja iedzīvotāji. Tomēr par šī vārda izcelsmi precīzu ziņu nav. Tas varēja rasties, atsaucoties uz mītisko Ziemeļeiropā it kā esošo Baltijas salu. Pastāv arī iespēja, ka Ādams no Brēmenes veidojis šo vārdu no ģermāņu vārda belt, ar kuru tiek apzīmēti vairāki Dānijas šaurumi. Cits skaidrojums — vārds cēlies no protoindoeiropiešu valodas vārda saknes *bhel, kas nozīmē ‘balts’, ‘mirdzošs’. Šī vārda sakne saglabājusies arī vairākās mūsdienu indoeiropiešu valodās, tai skaitā latviešu valodā. Vienā no senākajām kartēm, kurā attēlota mūsdienu Latvijas teritorija — Aleksandrijas zinātnieka Klaudija Ptolemaja (ap 90.—168. m. ē.) izveidotajā Austrumeiropas kartē Baltijas jūra nosaukta par "Sarmatu jūru" (MARE SARMATICVM). Citos avotos minēts nosaukums OCEANUS SARMATICUS. Sarmati bija sena klejotāju tauta, kura runājusi indoirāņu valodā un līdz 4. gadsimtam dzīvoja Austrumeiropā. Tacits to dēvēja par "Svēbu jūru" (Mare Suebicum). Vēsturē pazīstami arī citu tautu dotie Baltijas jūras nosaukumi: "Varjagu jūra", "Barbaru jūra",[2] "Vendu jūra". Tā kādā 946. gada dokumentā to dēvēja par "Rūgu jūru" (no rūgu vārda cēlies tagadējās Rīgenes salas nosaukums),[3] bet Nestora hronikas ievadā Baltijas jūra nosaukta par Varjagu jūru.[4][5] Štumpfa (Stumpf) Eiropas kartē,[6] kas izdota Cīrihē, 1548. gadā (pārkopēta no vecākas Sebastiana Minstera veidotas kartes), Baltijas jūra tiek saukta par "Vācu jūru" (Das Deutsche Meer). Arī vecākā 1450. gadā Fra Mauro zīmētajā pasaules kartē redzams nosaukums "Ģermāņu jūra" vai pat okeāns, ko parasti piedēvē Ziemeļjūrai, tomēr minētajā kartē tā nepārprotami ir arī Baltijas jūra.[7] Latvijas piekrastes iedzīvotāji mēdza atklāto jūru dēvēt par "Dižjūru", bet Rīgas līci — par "Mazo jūru" (Mazjūru). Viduslaikos Rīgas līci latīniski dēvēja par "Līvu jūru" (Mare Livonicum). Senākajās Eiropas kartēs par jūrām tika saukti arī lielākie Baltijas jūras līči. Piemēram, Mare Finonicum sive Sinus Venedicus — tagadējais Somu jūras līcis."""

doc = nlp(text)

# 先生成 token 表格
rows = []
for sent in doc.sents:
    for token in sent:
        rows.append({
            "Text": token.text,
            "Lemma": token.lemma_,
            "POS": token.pos_,
            "Dependency": token.dep_,
            "Head": token.head.text
        })

df = pd.DataFrame(rows)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
display(df)  # 显示 token 表格

# 再单独显示分句
print("\n✅ 分句结果：")
for i, sent in enumerate(doc.sents, 1):
    print(f"Sentence {i}: {sent.text}")

,Text,Lemma,POS,Dependency,Head
0,Baltijas,Baltijas,PROPN,nmod,jūras
1,jūras,jūra,NOUN,nmod,nosaukums
2,nosaukums,nosaukums,NOUN,nsubj,devis
3,ir,būt,AUX,aux,devis
4,devis,dot,VERB,ROOT,devis
5,nosaukumu,nosaukums,NOUN,obj,devis
6,baltu,balts,NOUN,amod,valodām
7,valodām,valoda,NOUN,nmod,nosaukumu
8,un,un,CCONJ,cc,valstīm
9,Baltijas,Baltijas,PROPN,nmod,valstīm



✅ 分句结果：
Sentence 1: Baltijas jūras nosaukums ir devis nosaukumu baltu valodām un Baltijas valstīm.
Sentence 2: Terminu "Baltijas jūra" (Mare Balticum) pirmoreiz lietoja vācu hronists Brēmenes Ādams 11. gadsimtā.
Sentence 3: Viņš, kā jau vietējais hronists, pierakstījis nosaukumu, kuru lietoja iedzīvotāji.
Sentence 4: Tomēr par šī vārda izcelsmi precīzu ziņu nav.
Sentence 5: Tas varēja rasties, atsaucoties uz mītisko Ziemeļeiropā it kā esošo Baltijas salu.
Sentence 6: Pastāv arī iespēja, ka Ādams no Brēmenes veidojis šo vārdu no ģermāņu vārda belt, ar kuru tiek apzīmēti vairāki Dānijas šaurumi.
Sentence 7: Cits skaidrojums — vārds cēlies no protoindoeiropiešu valodas vārda saknes *bhel, kas nozīmē ‘balts’, ‘mirdzošs’.
Sentence 8: Šī vārda sakne saglabājusies arī vairākās mūsdienu indoeiropiešu valodās, tai skaitā latviešu valodā.
Sentence 9: Vienā no senākajām kartēm, kurā attēlota mūsdienu Latvijas teritorija — Aleksandrijas zinātnieka Klaudija Ptolemaja (ap 90.—168. m.
Sentence 10: ē

In [14]:
## Usage
## 使用方法

import spacy
import numpy as np

# Load the pipeline
# 加载模型流水线
nlp = spacy.load("lv_spacy_cnn")

# Example text
# 示例文本
text = """Baltijas jūras nosaukums ir devis nosaukumu baltu valodām un Baltijas valstīm.
Terminu "Baltijas jūra" (Mare Balticum) pirmoreiz lietoja vācu hronists Brēmenes Ādams 11. gadsimtā."""

# Process text
# 处理文本
doc = nlp(text)

# ------------------------
# Tokenization / 分词
# ------------------------
print("Tokens / 分词结果:")
print([token.text for token in doc])

# ------------------------
# Lemmatization / 词形还原
# ------------------------
print("Lemmas / 词形还原结果:")
print([token.lemma_ for token in doc])

# ------------------------
# Part-of-Speech Tagging / 词性标注
# ------------------------
print("POS tags / 词性标注:")
for token in doc:
    print(f"{token.text}: {token.pos_} ({token.tag_})")
# pos_ 是 Universal POS 标签, tag_ 是语言特定 POS 标签

# ------------------------
# Morphological Features / 形态特征
# ------------------------
print("Morphological features / 形态特征:")
for token in doc:
    print(f"{token.text}: {token.morph}")
# 输出形态信息，如格、数、性别、时态等

# ------------------------
# Dependency Parsing / 依存句法分析
# ------------------------
print("Dependency parsing / 依存句法分析:")
for token in doc:
    print(f"{token.text} <--{token.dep_}-- {token.head.text}")
# 输出每个 token 的依存关系及其父节点

# ------------------------
# Sentence Segmentation / 分句
# ------------------------
print("Sentences / 分句结果:")
for sent in doc.sents:
    print(sent.text)

# ------------------------
# 直接访问流水线组件（可选，高级用法）
# ------------------------
print("Pipeline components / 流水线组件:")
print(nlp.pipe_names)

# Tok2Vec (词向量表示)
vectors = np.vstack([token.vector for token in doc])
print("Token vectors shape / Token 向量维度:", vectors.shape)


Tokens / 分词结果:
['Baltijas', 'jūras', 'nosaukums', 'ir', 'devis', 'nosaukumu', 'baltu', 'valodām', 'un', 'Baltijas', 'valstīm', '.', '\n', 'Terminu', '"', 'Baltijas', 'jūra', '"', '(', 'Mare', 'Balticum', ')', 'pirmoreiz', 'lietoja', 'vācu', 'hronists', 'Brēmenes', 'Ādams', '11', '.', 'gadsimtā', '.']
Lemmas / 词形还原结果:
['Baltijas', 'jūra', 'nosaukums', 'būt', 'dot', 'nosaukums', 'balts', 'valoda', 'un', 'Baltijas', 'valsts', '.', '\n', 'Terminu', '"', 'Baltijas', 'jūra', '"', '(', 'Mare', 'Balticum', ')', 'pirmoreiz', 'lietot', 'vāci', 'hronists', 'Brēmenes', 'Ādams', '11', '.', 'gadsimts', '.']
POS tags / 词性标注:
Baltijas: PROPN (npfsg4)
jūras: NOUN (ncfsg4)
nosaukums: NOUN (ncmsn1)
ir: AUX (vcnipii30an)
devis: VERB (vmnpdmsnasnpn)
nosaukumu: NOUN (ncmsa1)
baltu: NOUN (ncmpg1)
valodām: NOUN (ncfpd4)
un: CCONJ (cc)
Baltijas: PROPN (npfsg4)
valstīm: NOUN (ncfpd6)
.: PUNCT (zs)

: VERB (vmnpdmpdapypn)
Terminu: NOUN (npmsa1)
": PUNCT (zq)
Baltijas: PROPN (npfsg4)
jūra: NOUN (ncfsn4)
": PUNCT 